In [21]:
import json
import pybedtools
import pandas

"""
In order to create a HiPiler configuration we need 4 things:

1. A list of 2D features, i.e., genomic loci (BED, BEDPE, etc.)
2. Configure the snippets
3. The data set UUID of the Hi-C map we want to explore
4. A HiPiler compatible HiGlass configuration

We will go over every step now.
"""

'\nIn order to create a HiPiler configuration we need 3 things:\n\n1. A list of 2D features, i.e., genomic loci (BED, BEDPE, etc.)\n2. The data set UUID of the Hi-C map we want to explore\n3. A HiPiler compatible HiGlass configuration\n\nWe will go over every step now.\n'

In [41]:
# Utility methods we are going to use

COL_NAMES_BED = [
    'chrom1',
    'start1',
    'end1',
    '_name',
    'score',
    'strand1'
]

COL_NAMES_BEDPE = [
    'chrom1',
    'start1',
    'end1',
    'chrom2',
    'start2',
    'end2',
    '_name',
    'score',
    'strand1',
    'strand2'
]

def df_to_json(df):
    """Convert Pandas DataFrame into HiPiler compatible JSON
    """
    return json.dumps([list(df.columns)] + df.values.tolist())

def bed_to_df(bed, *args, **kwargs):
    """Convert BED into Pandas DataFrame
    """
    # Complain if BAM or if not a file
    if bed._isbam:
        raise ValueError("BAM not supported for converting to DataFrame")
    if not isinstance(bed.fn, basestring):
        raise ValueError("use .saveas() to make sure bed.fn is a file")

    # Otherwise we're good:
    return pandas.read_table(bed.fn, *args, **kwargs)

def bed_to_hipiler(bed, ds, zoom):
    """Convert BED into HiPiler compatible JSON
    """
    
    df = bed_to_df(bed, names=COL_NAMES_BED)
    
    # Copy columns because HiPiler requires 2-dimensional information
    df['chrom2'] = df['chrom1']
    df['start2'] = df['start1']
    df['end2'] = df['end1']
    df['strand2'] = df['strand1']
    
    # Add the data set UUID from HiGlass server
    df['dataset'] = ds
    
    # Finally add the zoom-out level
    df['zoomoutlevel'] = zoom
    
    return df_to_json(df)

def bedpe_to_hipiler(bedpe, ds, zoom, extra_meas=[], extra_cats=[]):
    """Convert BED into HiPiler compatible JSON
    """
    
    names = COL_NAMES_BEDPE + extra_meas + map(lambda cat: '_' + cat, extra_cats)
    
    df = bed_to_df(bedpe, names=names)
    
    # Add the data set UUID from HiGlass server
    df['dataset'] = ds
    
    # Finally add the zoom-out level
    df['zoomoutlevel'] = zoom
    
    return df_to_json(df)

def df_to_hipiler(df):
    required_columns = ['chrom1', 'start1', 'end1', '']
    
    df[['A', 'B']]
    
    return df_to_json()

In [43]:
"""
1. Step: Get a Pandas DataFrame of Features

Either load a BED or BEDPE file directly or create a DataFrame manually.
You can use `bed_to_df` with `COL_NAMES_BED` or `COL_NAMES_BEDPE` to automatically
assign the required column names for HiPiler or do that yourself.

Take a look at https://github.com/flekschas/hipiler/wiki/Data#bedpe-like-table
to find out more about the required column names.

Any additional column is treated as a numerical measurement unless prefixed with an underscore (`_`).
Columns prefixed with an underscore are treated as categorical values.
"""

# Load a BED file
df = bed_to_df(pybedtools.example_bedtool('a.bed'), names=COL_NAMES_BED)

# 1B. Load a BEDPE file
df = bed_to_df(pybedtools.example_bedtool('bedpe.bed'), names=COL_NAMES_BEDPE)

# Load a CSV file with Pandas
df = pandas.read_csv()

https://github.com/flekschas/hipiler/wiki/Data#config-file

,chrom1,start1,end1,chrom2,start2,end2,_name,score,strand1,strand2
0,chr1,100,200,chr5,5000,5100,bedpe_example1,30,+,-
1,chr9,1000,5000,chr9,3000,3800,bedpe_example2,100,-,-


In [ ]:
"""
2. Step: Configure the Snippets

Most settings are optional but we 

For additional information take a look at:

https://github.com/flekschas/hipiler/wiki/Data#config-file
"""

# URL to a HiGlass server
server = 'http://higlass.io'

# Float precision of the raw matrix snippets to be returned by the server
precision = 2
    
# Pixel dimension of the matrix snippets toll be returned by the server. Since HiPiler currently only supports squared snippets only one dimension is needed.
dims = 50

# Number of bins used for padding. This only matters when loading domains.
padding = 

# If true disables loading snippets from cache. Good for testing.
noCache = False

# If true disables Cooler's balancing.
noBalance = True

# Percentile at which the values are capped.
percentile = 99

# Number of diagonals that are ignored, i.e., set to zero.
ignoreDiags = 1

In [ ]:
"""
3. Step: Get the data set's UUID.

Go to http://your.higlass.server.com/api/v1/tilesets/?limit=1000
and search (STRG + F or CMD + F) for your data set and copy the uuid.
"""

ds = ''

In [15]:
bed_to_hipiler(bed)

'[["chrom1", "start1", "end1", "_name1", "score1", "strand1", "chrom2", "start2", "end2", "strand2", "_name2", "score2"], ["chr1", 1, 100, "feature1", 0, "+", "chr1", 1, 100, "+", "feature1", 0], ["chr1", 100, 200, "feature2", 0, "+", "chr1", 100, 200, "+", "feature2", 0], ["chr1", 150, 500, "feature3", 0, "-", "chr1", 150, 500, "-", "feature3", 0], ["chr1", 900, 950, "feature4", 0, "+", "chr1", 900, 950, "+", "feature4", 0]]'

In [28]:
bedpe_to_hipiler(pybedtools.example_bedtool('bedpe.bed'), 'ass', 0)

'[["chrom1", "start1", "end1", "chrom2", "start2", "end2", "_name", "score", "strand1", "strand2", "dataset", "zoomoutlevel"], ["chr1", 100, 200, "chr5", 5000, 5100, "bedpe_example1", 30, "+", "-", "ass", 0], ["chr9", 1000, 5000, "chr9", 3000, 3800, "bedpe_example2", 100, "-", "-", "ass", 0]]'

In [40]:
print pybedtools.example_bedtool('bedpe.bed')._isbedpe

AttributeError: 'BedTool' object has no attribute '_isbedpe'